In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Academics/Sem8/DataMining/CovidDataset.csv')

In [ ]:
item_list = list(df.columns)
item_dict = dict()

for i, item in enumerate(item_list):
    item_dict[item] = i + 1

item_dict

{'Breathing Problem': 1,
 'Fever': 2,
 'Dry Cough': 3,
 'Sore throat': 4,
 'Running Nose': 5,
 'Asthma': 6,
 'Chronic Lung Disease': 7,
 'Headache': 8,
 'Heart Disease': 9,
 'Diabetes': 10,
 'Hyper Tension': 11,
 'Fatigue ': 12,
 'Gastrointestinal ': 13,
 'Abroad travel': 14,
 'Contact with COVID Patient': 15,
 'Attended Large Gathering': 16,
 'Visited Public Exposed Places': 17,
 'Family working in Public Exposed Places': 18,
 'Wearing Masks': 19,
 'Sanitization from Market': 20,
 'COVID-19': 21}

In [ ]:
transactions = list()

for i, row in df.iterrows():
    transaction = set()

    for item in item_dict:
        if row[item] == 'Yes':
            transaction.add(item_dict[item])
    transactions.append(transaction)

transactions

[{1, 2, 3, 4, 5, 10, 11, 12, 13, 15, 17, 18, 21},
 {1, 2, 3, 4, 6, 7, 8, 12, 16, 17, 21},
 {1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 21},
 {1, 2, 3, 6, 9, 10, 14, 16, 17, 21},
 {1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 15, 17, 21},
 {1, 2, 3, 9, 11, 21},
 {1, 2, 3, 7, 9, 10, 11, 12, 13, 16, 17, 18, 21},
 {1, 2, 3, 5, 6, 10, 11, 13, 14, 17, 21},
 {1, 2, 3, 5, 7, 10, 12, 14, 15, 16, 21},
 {1, 2, 3, 6, 10, 11, 12, 17, 21},
 {1, 2, 3, 7, 9, 11, 14, 16, 21},
 {1, 2, 3, 4, 5, 6, 9, 10, 11, 13, 14, 16, 18, 21},
 {1, 2, 3, 4, 6, 7, 11, 13, 14, 17, 18, 21},
 {1, 2, 3, 4, 6, 10, 11, 14, 15, 18, 21},
 {1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 15, 16, 17, 18, 21},
 {1, 2, 3, 5, 8, 9, 21},
 {1, 2, 3, 5, 8, 9, 13, 15, 18, 21},
 {1, 2, 3, 5, 6, 7, 8, 9, 13, 14, 17, 21},
 {1, 2, 3, 4, 7, 8, 9, 10, 12, 18, 21},
 {1, 2, 3, 4, 6, 8, 10, 11, 12, 13, 14, 17, 18, 21},
 {1, 2, 3, 5, 8, 10, 11, 12, 14, 21},
 {1, 2, 3, 5, 6, 7, 11, 12, 14, 15, 21},
 {1, 2, 3, 5, 7, 10, 12, 18, 21},
 {1, 2, 3, 5, 6, 8, 11, 12, 16, 17, 21},


In [ ]:
def get_support(transactions, item_set):
    match_count = 0
    for transaction in transactions:
        if item_set.issubset(transaction):
            match_count += 1

    return float(match_count/len(transactions))

In [ ]:
def self_join(frequent_item_sets_per_level, level):
    current_level_candidates = list()
    last_level_items = frequent_item_sets_per_level[level - 1]

    if len(last_level_items) == 0:
        return current_level_candidates

    for i in range(len(last_level_items)):
        for j in range(i+1, len(last_level_items)):
            itemset_i = last_level_items[i][0]
            itemset_j = last_level_items[j][0]
            union_set = itemset_i.union(itemset_j)

            if union_set not in current_level_candidates and len(union_set) == level:
                current_level_candidates.append(union_set)

    return current_level_candidates

In [ ]:
def get_single_drop_subsets(item_set):
    single_drop_subsets = list()
    for item in item_set:
        temp = item_set.copy()
        temp.remove(item)
        single_drop_subsets.append(temp)

    return single_drop_subsets

def is_valid_set(item_set, prev_level_sets):
    single_drop_subsets = get_single_drop_subsets(item_set)

    for single_drop_set in single_drop_subsets:
        if single_drop_set not in prev_level_sets:
            return False
    return True

def pruning(frequent_item_sets_per_level, level, candidate_set):
    post_pruning_set = list()
    if len(candidate_set) == 0:
        return post_pruning_set

    prev_level_sets = list()
    for item_set, _ in frequent_item_sets_per_level[level - 1]:
        prev_level_sets.append(item_set)

    for item_set in candidate_set:
        if is_valid_set(item_set, prev_level_sets):
            post_pruning_set.append(item_set)

    return post_pruning_set

In [ ]:
from collections import defaultdict

def apriori(min_support):
    frequent_item_sets_per_level = defaultdict(list)
    print("level : 1", end = " ")

    for item in range(1, len(item_list) + 1):
        support = get_support(transactions, {item})
        if support >= min_support:
            frequent_item_sets_per_level[1].append(({item}, support))

    for level in range(2, len(item_list) + 1):
        print(level, end = " ")
        current_level_candidates = self_join(frequent_item_sets_per_level, level)

        post_pruning_candidates = pruning(frequent_item_sets_per_level, level, current_level_candidates)
        if len(post_pruning_candidates) == 0:
            break

        for item_set in post_pruning_candidates:
            support = get_support(transactions, item_set)
            if support >= min_support:
                frequent_item_sets_per_level[level].append((item_set, support))

    return frequent_item_sets_per_level

In [ ]:
min_support = 0.50
frequent_item_sets_per_level = apriori(min_support)

level : 1 2 3 4 5 

In [ ]:
for level in frequent_item_sets_per_level:
    print(len(frequent_item_sets_per_level[level]))

10
10
7
1


In [ ]:
for level in frequent_item_sets_per_level:
    print(frequent_item_sets_per_level[level])

[({1}, 0.6661759293338241), ({2}, 0.7863452337136547), ({3}, 0.7926021347073978), ({4}, 0.7274567537725433), ({5}, 0.5432462274567538), ({8}, 0.5034965034965035), ({12}, 0.5191387559808612), ({15}, 0.5016562384983437), ({17}, 0.5189547294810453), ({21}, 0.8065881486934119)]
[({1, 2}, 0.5412219359587781), ({1, 3}, 0.5585204269414795), ({1, 4}, 0.548398969451601), ({1, 21}, 0.6199852778800147), ({2, 3}, 0.6444608023555392), ({2, 4}, 0.6308428413691571), ({2, 21}, 0.6913875598086124), ({3, 4}, 0.6152005888847994), ({3, 21}, 0.7136547662863453), ({4, 21}, 0.6751932278248067)]
[({1, 2, 21}, 0.5099374309900626), ({1, 3, 21}, 0.5301803459698197), ({1, 4, 21}, 0.5255796834744203), ({2, 3, 4}, 0.5261317629738682), ({2, 3, 21}, 0.611152005888848), ({2, 4, 21}, 0.5964298859035702), ({3, 4, 21}, 0.5933014354066986)]
[({2, 3, 4, 21}, 0.5191387559808612)]


In [ ]:
item_support_dict = dict()
item_list = list()

key_list = list(item_dict.keys())
val_list = list(item_dict.values())

for level in frequent_item_sets_per_level:
    for set_support_pair in frequent_item_sets_per_level[level]:
        for i in set_support_pair[0]:
            item_list.append(key_list[val_list.index(i)])
        item_support_dict[frozenset(item_list)] = set_support_pair[1]
        item_list = list()

In [ ]:
item_support_dict

{frozenset({'Breathing Problem'}): 0.6661759293338241,
 frozenset({'Fever'}): 0.7863452337136547,
 frozenset({'Dry Cough'}): 0.7926021347073978,
 frozenset({'Sore throat'}): 0.7274567537725433,
 frozenset({'Running Nose'}): 0.5432462274567538,
 frozenset({'Headache'}): 0.5034965034965035,
 frozenset({'Fatigue '}): 0.5191387559808612,
 frozenset({'Contact with COVID Patient'}): 0.5016562384983437,
 frozenset({'Visited Public Exposed Places'}): 0.5189547294810453,
 frozenset({'COVID-19'}): 0.8065881486934119,
 frozenset({'Breathing Problem', 'Fever'}): 0.5412219359587781,
 frozenset({'Breathing Problem', 'Dry Cough'}): 0.5585204269414795,
 frozenset({'Breathing Problem', 'Sore throat'}): 0.548398969451601,
 frozenset({'Breathing Problem', 'COVID-19'}): 0.6199852778800147,
 frozenset({'Dry Cough', 'Fever'}): 0.6444608023555392,
 frozenset({'Fever', 'Sore throat'}): 0.6308428413691571,
 frozenset({'COVID-19', 'Fever'}): 0.6913875598086124,
 frozenset({'Dry Cough', 'Sore throat'}): 0.615200

In [ ]:
def find_subset(item, item_length):
    combs = []
    for i in range(1, item_length + 1):
        combs.append(list(combinations(item, i)))

    subsets = []
    for comb in combs:
        for elt in comb:
            subsets.append(elt)

    return subsets

In [ ]:
def association_rules(min_confidence, support_dict):
    rules = list()
    for item, support in support_dict.items():
        item_length = len(item)

        if item_length > 1:
            subsets = find_subset(item, item_length)

            for A in subsets:
                B = item.difference(A)

                if B:
                    A = frozenset(A)

                    AB = A | B

                    confidence = support_dict[AB] / support_dict[A]
                    if confidence >= min_confidence:
                        rules.append((A, B, confidence))

    return rules

In [ ]:
association_rules = association_rules(min_confidence = 0.5, support_dict = item_support_dict)

In [ ]:
print("Number of rules: ", len(association_rules), "\n")

for rule in association_rules:
    print('{0} -> {1} <confidence: {2}>'.format(set(rule[0]), set(rule[1]), rule[2]))

Number of rules:  76 

{'Breathing Problem'} -> {'Fever'} <confidence: 0.8124309392265194>
{'Fever'} -> {'Breathing Problem'} <confidence: 0.6882752164755442>
{'Breathing Problem'} -> {'Dry Cough'} <confidence: 0.8383977900552485>
{'Dry Cough'} -> {'Breathing Problem'} <confidence: 0.7046668214534478>
{'Breathing Problem'} -> {'Sore throat'} <confidence: 0.8232044198895028>
{'Sore throat'} -> {'Breathing Problem'} <confidence: 0.7538578294965849>
{'Breathing Problem'} -> {'COVID-19'} <confidence: 0.9306629834254143>
{'COVID-19'} -> {'Breathing Problem'} <confidence: 0.7686516084873374>
{'Fever'} -> {'Dry Cough'} <confidence: 0.819564708635619>
{'Dry Cough'} -> {'Fever'} <confidence: 0.8130949616902716>
{'Fever'} -> {'Sore throat'} <confidence: 0.8022466651064826>
{'Sore throat'} -> {'Fever'} <confidence: 0.8671894763470781>
{'COVID-19'} -> {'Fever'} <confidence: 0.8571754506046086>
{'Fever'} -> {'COVID-19'} <confidence: 0.8792417505265622>
{'Sore throat'} -> {'Dry Cough'} <confidence: 